# Tên đề tài:  Ứng dụng học máy trong bài toán phát hiện lỗi từ log phần mềm

## Đọc dữ liệu từ Kaggle

In [ ]:
# Cài đặt package kaggle
!pip install -q kaggle

In [ ]:
# Tải file kaggle.json để xác thực
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akaichi21","key":"dba640f842cd90630c188e6f3985eafc"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Tải dữ liệu lên
!kaggle datasets download -d mehulkatara/windows-event-log

Dataset URL: https://www.kaggle.com/datasets/mehulkatara/windows-event-log
License(s): unknown
  0% 0.00/6.80M [00:00<?, ?B/s]
100% 6.80M/6.80M [00:00<00:00, 72.3MB/s]


In [ ]:
# Giải nén dữ liệu
import zipfile
zip_ref = zipfile.ZipFile('windows-event-log.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

## Các thư viện cần thiết

In [ ]:
!pip install pyspellchecker

In [ ]:
import numpy as np
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm

from sklearn.metrics import accuracy_score, classification_report

## Tiền xử lý dữ liệu

In [ ]:
# Dữ liệu liên quan đến log của sự kiện phần mềm hệ thống Window
df = pd.read_csv('eventlog.csv')
df.head()

,Unnamed: 0,MachineName,Category,EntryType,Message,Source,TimeGenerated,country,regionName,city,zip,timezone,isp
0,1,LAPTOP-1MKMTVPM,(0),Information,Successfully scheduled Software Protection ser...,Software Protection Platform Service,2020-11-14 08:41:59,India,Gujarat,Ahmedabad,380007,Asia/Kolkata,GTPL A. J. Enterprise
1,2,LAPTOP-1MKMTVPM,Logging/Recovery,Error,"svchost (13360,R,98) TILEREPOSITORYS-1-5-18: E...",ESENT,2020-11-14 08:25:14,India,Gujarat,Ahmedabad,380007,Asia/Kolkata,GTPL A. J. Enterprise
2,3,LAPTOP-1MKMTVPM,Logging/Recovery,Error,"svchost (15040,R,98) TILEREPOSITORYS-1-5-18: E...",ESENT,2020-11-14 08:35:10,India,Gujarat,Ahmedabad,380007,Asia/Kolkata,GTPL A. J. Enterprise
3,4,LAPTOP-1MKMTVPM,(0),Information,Successfully scheduled Software Protection ser...,Software Protection Platform Service,2020-11-14 08:45:38,India,Gujarat,Ahmedabad,380007,Asia/Kolkata,GTPL A. J. Enterprise
4,5,LAPTOP-1MKMTVPM,(0),Information,Successfully created restore point (Process = ...,System Restore,2020-11-14 08:22:34,India,Gujarat,Ahmedabad,380007,Asia/Kolkata,GTPL A. J. Enterprise


In [ ]:
# Xóa các cột không cần thiết
df = df.drop(['country', 'regionName', 'city', 'zip', 'timezone', 'isp'], axis=1)
df.head()

,Unnamed: 0,MachineName,Category,EntryType,Message,Source,TimeGenerated
0,1,LAPTOP-1MKMTVPM,(0),Information,Successfully scheduled Software Protection ser...,Software Protection Platform Service,2020-11-14 08:41:59
1,2,LAPTOP-1MKMTVPM,Logging/Recovery,Error,"svchost (13360,R,98) TILEREPOSITORYS-1-5-18: E...",ESENT,2020-11-14 08:25:14
2,3,LAPTOP-1MKMTVPM,Logging/Recovery,Error,"svchost (15040,R,98) TILEREPOSITORYS-1-5-18: E...",ESENT,2020-11-14 08:35:10
3,4,LAPTOP-1MKMTVPM,(0),Information,Successfully scheduled Software Protection ser...,Software Protection Platform Service,2020-11-14 08:45:38
4,5,LAPTOP-1MKMTVPM,(0),Information,Successfully created restore point (Process = ...,System Restore,2020-11-14 08:22:34


In [ ]:
# Đổi tên các cột dữ liệu
df.columns = ['id', 'machine_name', 'task_category', 'level', 'message', 'source', 'timestamp']
df.head()

,id,machine_name,task_category,level,message,source,timestamp
0,1,LAPTOP-1MKMTVPM,(0),Information,Successfully scheduled Software Protection ser...,Software Protection Platform Service,2020-11-14 08:41:59
1,2,LAPTOP-1MKMTVPM,Logging/Recovery,Error,"svchost (13360,R,98) TILEREPOSITORYS-1-5-18: E...",ESENT,2020-11-14 08:25:14
2,3,LAPTOP-1MKMTVPM,Logging/Recovery,Error,"svchost (15040,R,98) TILEREPOSITORYS-1-5-18: E...",ESENT,2020-11-14 08:35:10
3,4,LAPTOP-1MKMTVPM,(0),Information,Successfully scheduled Software Protection ser...,Software Protection Platform Service,2020-11-14 08:45:38
4,5,LAPTOP-1MKMTVPM,(0),Information,Successfully created restore point (Process = ...,System Restore,2020-11-14 08:22:34


In [ ]:
# Xem kích thước dữ liệu
len(df)

158184

In [ ]:
# Kiểm tra dữ liệu bị rỗng
df.isnull().sum()

,0
id,0
machine_name,0
task_category,0
level,0
message,0
source,0
timestamp,0


In [ ]:
# Kiểm tra các giá trị trùng lặp
df.duplicated().sum()

0

In [ ]:
# Kiểm tra các tham số khác biệt của cột 'level'
df['level'].value_counts()

,count
level,
Information,138489
Warning,9911
Error,5322
0,4462


In [ ]:
# Xóa các dòng dữ liệu có cột 'level' là 0
df = df.loc[df['level'] != '0']
print(len(df))
print(df['level'].value_counts())

153722
level
Information    138489
Warning          9911
Error            5322
Name: count, dtype: int64


In [ ]:
# Kiểm tra các tham số khác biệt của cột 'machine_name'
df['machine_name'].value_counts()

,count
machine_name,
LAPTOP-1MKMTVPM,75710
TMP249-G3-M,47007
Mehul,9971
DESKTOP-SEJ28PM,8183
DESKTOP-U66O8IT,4877
Admin-PC,4685
DESKTOP-R5JMQHG,3253
WIN-3EDLS10RKSL,21
ADMIN-PC,15


In [ ]:
# Chỉ giữ lại các dòng dữ liệu có tên máy tính cụ thể
df = df.loc[~df['machine_name'].isin(['Mehul', 'Admin-PC', 'ADMIN-PC'])]
print(len(df))
print(df['machine_name'].value_counts())

139051
machine_name
LAPTOP-1MKMTVPM    75710
TMP249-G3-M        47007
DESKTOP-SEJ28PM     8183
DESKTOP-U66O8IT     4877
DESKTOP-R5JMQHG     3253
WIN-3EDLS10RKSL       21
Name: count, dtype: int64


In [ ]:
# Kiểm tra các tham số khác biệt của cột 'task_category'
df['task_category'].value_counts()

,count
task_category,
(0),104619
General,19431
Logging/Recovery,10444
(1),2620
Search service,842
LMS,272
Application Crashing Events,248
GENERAL,170
Performance,139


In [ ]:
# Xóa các dòng dữ liệu có cột 'task_category' không có tên loại tác vụ cụ thể
df = df.loc[~df['task_category'].isin(['(0)', '(1)', '(101)', '(18)',
                                        '(2)', '(14)', '(3)'])]
print(len(df))
print(df['task_category'].value_counts())

31675
task_category
General                          19431
Logging/Recovery                 10444
Search service                     842
LMS                                272
Application Crashing Events        248
GENERAL                            170
Performance                        139
Table/Column/Index Definition      103
Gatherer                            26
Name: count, dtype: int64


In [ ]:
# Kiểm tra các tham số khác biệt của cột 'source'
df['source'].value_counts()

,count
source,
ESENT,30117
Windows Search Service,868
LMS,272
Application Error,248
DPTF,170


In [ ]:
# Xử lý dữ liệu định dạng thời gian
# Chuyển đổi timestamp thành định dạng datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Tạo các đặc trưng thời gian
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.day_name()

# Hiển thị kết quả
print(df[['timestamp', 'hour', 'day_of_week']])

                 timestamp  hour day_of_week
1      2020-11-14 08:25:14     8    Saturday
2      2020-11-14 08:35:10     8    Saturday
6      2020-11-14 07:21:31     7    Saturday
12     2020-11-14 07:21:31     7    Saturday
15     2020-11-14 07:21:31     7    Saturday
...                    ...   ...         ...
158133 2021-03-10 16:46:24    16   Wednesday
158141 2021-03-08 06:59:31     6      Monday
158158 2021-03-10 16:46:43    16   Wednesday
158159 2021-03-10 16:46:23    16   Wednesday
158169 2021-03-08 07:08:51     7      Monday

[31675 rows x 3 columns]


In [ ]:
# Chuyển cột 'day_of_week' về giá trị số
day_of_week_mapping = {
    'Sunday': 0,
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
}
df['day_of_week'] = df['day_of_week'].map(day_of_week_mapping)

# Hiển thị kết quả
print(df[['day_of_week']])

        day_of_week
1                 6
2                 6
6                 6
12                6
15                6
...             ...
158133            3
158141            1
158158            3
158159            3
158169            1

[31675 rows x 1 columns]


In [ ]:
# Sử dụng mã hóa One-hot cho các trường phân loại
df_encoded = pd.get_dummies(df, columns=['machine_name', 'task_category', 'source'])

# Hiển thị kết quả
print(df_encoded.head())

    id        level                                            message  \
1    2        Error  svchost (13360,R,98) TILEREPOSITORYS-1-5-18: E...   
2    3        Error  svchost (15040,R,98) TILEREPOSITORYS-1-5-18: E...   
6    7  Information  Video.UI (13712,D,50) {6F733DD1-ACD6-456C-8F90...   
12  13  Information  Video.UI (13712,R,98) {6F733DD1-ACD6-456C-8F90...   
15  16  Information  Video.UI (13712,D,50) {6F733DD1-ACD6-456C-8F90...   

             timestamp  hour  day_of_week  machine_name_DESKTOP-R5JMQHG  \
1  2020-11-14 08:25:14     8            6                         False   
2  2020-11-14 08:35:10     8            6                         False   
6  2020-11-14 07:21:31     7            6                         False   
12 2020-11-14 07:21:31     7            6                         False   
15 2020-11-14 07:21:31     7            6                         False   

    machine_name_DESKTOP-SEJ28PM  machine_name_DESKTOP-U66O8IT  \
1                          False      

In [ ]:
# Xử lý các dòng dữ liệu trong cột 'message'

# Tải các stopwords và WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
spell = SpellChecker()
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def is_noise_word(word, max_repeating_chars=3, min_length=2, max_length=15):
    # Kiểm tra ký tự lặp lại liên tiếp
    if re.search(r'(.)\1{%d,}' % max_repeating_chars, word):
        return True
    # Kiểm tra độ dài của từ
    if len(word) < min_length or len(word) > max_length:
        return True
    # Kiểm tra từ điển chính tả
    if word not in spell.word_frequency:
        return True
    return False

def clean_text(text):
    # Chuyển về chữ thường
    text = text.lower()
    # Xóa ký tự đặc biệt và dấu câu
    text = re.sub(r'[^a-z\s]', '', text)
    # Loại bỏ các từ không hợp lệ
    words = text.split()
    words = [word for word in words if not is_noise_word(word)]
    # Loại bỏ stop words và lemmatization
    cleaned_text = ' '.join(lemmatizer.lemmatize(word) for word in words if word not in stop_words)
    return cleaned_text

df['message'] = df['message'].apply(clean_text)

# Sử dụng TF-IDF để vector hóa cho cột 'message'
vectorizer = TfidfVectorizer(max_features=1000)
message_vectors = vectorizer.fit_transform(df['message'])

# Chuyển đổi sparse matrix thành DataFrame
message_df = pd.DataFrame(message_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Hiển thị kết quả
print(message_df.head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


   abnormally  access  accessed  ace  action   ad  add  addition  additional  \
0         0.0     0.0       0.0  0.0     0.0  0.0  0.0       0.0         0.0   
1         0.0     0.0       0.0  0.0     0.0  0.0  0.0       0.0         0.0   
2         0.0     0.0       0.0  0.0     0.0  0.0  0.0       0.0         0.0   
3         0.0     0.0       0.0  0.0     0.0  0.0  0.0       0.0         0.0   
4         0.0     0.0       0.0  0.0     0.0  0.0  0.0       0.0         0.0   

   administrator  ...      used  user  validating  value  vendor  verify  \
0            0.0  ...  0.000000   0.0         0.0    0.0     0.0     0.0   
1            0.0  ...  0.000000   0.0         0.0    0.0     0.0     0.0   
2            0.0  ...  0.000000   0.0         0.0    0.0     0.0     0.0   
3            0.0  ...  0.000000   0.0         0.0    0.0     0.0     0.0   
4            0.0  ...  0.310079   0.0         0.0    0.0     0.0     0.0   

    version  whether  window  write  
0  0.000000      0.0    

In [ ]:
column_list = message_df.columns.tolist()
print(column_list)

['abnormally', 'access', 'accessed', 'ace', 'action', 'ad', 'add', 'addition', 'additional', 'administrator', 'ago', 'also', 'appear', 'application', 'assistance', 'attached', 'attempting', 'back', 'backup', 'bad', 'bed', 'build', 'byte', 'cache', 'cad', 'cannot', 'catalog', 'catalogue', 'check', 'cleanup', 'click', 'closed', 'code', 'command', 'commit', 'completed', 'computer', 'configuration', 'connection', 'contact', 'contacted', 'container', 'content', 'context', 'controlled', 'copy', 'corrects', 'corrupt', 'corrupted', 'could', 'crawl', 'create', 'created', 'creating', 'current', 'customer', 'dad', 'damaged', 'data', 'database', 'date', 'db', 'dc', 'de', 'deb', 'default', 'deferred', 'delete', 'deleted', 'detached', 'detail', 'detected', 'determine', 'diagnosing', 'digital', 'dirty', 'disk', 'domain', 'drive', 'driver', 'due', 'dynamic', 'eff', 'engine', 'enter', 'err', 'error', 'event', 'example', 'exception', 'excluded', 'executing', 'fa', 'fab', 'failed', 'fault', 'faulting', '

In [ ]:
# Kết hợp các DataFrame sau khi tiền xử lý
df_final = pd.concat([df_encoded.reset_index(drop=True), message_df.reset_index(drop=True)], axis=1)

# Loại bỏ các cột không còn cần thiết
df_final.drop(columns=['message', 'timestamp'], inplace=True)

# Hiển thị kết quả cuối cùng
print(df_final.head())

   id        level  hour  day_of_week  machine_name_DESKTOP-R5JMQHG  \
0   2        Error     8            6                         False   
1   3        Error     8            6                         False   
2   7  Information     7            6                         False   
3  13  Information     7            6                         False   
4  16  Information     7            6                         False   

   machine_name_DESKTOP-SEJ28PM  machine_name_DESKTOP-U66O8IT  \
0                         False                         False   
1                         False                         False   
2                         False                         False   
3                         False                         False   
4                         False                         False   

   machine_name_LAPTOP-1MKMTVPM  machine_name_TMP249-G3-M  \
0                          True                     False   
1                          True                     False   

In [ ]:
# Kiểm tra các tham số khác biệt của cột 'level'
df_final['level'].value_counts()

,count
level,
Information,26710
Error,2757
Warning,2208


In [ ]:
# Chuyển cột 'level' về giá trị số
level_mapping = {
    'Information': 0,
    'Warning': 1,
    'Error': 2,
}

df_final['level'] = df_final['level'].map(level_mapping)

# Hiển thị kết quả
print(df_final[['level']])

       level
0          2
1          2
2          0
3          0
4          0
...      ...
31670      0
31671      0
31672      1
31673      0
31674      1

[31675 rows x 1 columns]


In [ ]:
# Tách đã các tập dữ liệu X và y
X = df_final.drop(columns=['level']) # Chứa các thuộc tính
y = df_final['level'] # Chứa nhãn

In [ ]:
numeric_columns = X.select_dtypes(include=['float64', 'int64', 'int32']).columns

In [ ]:
numeric_columns

Index(['id', 'hour', 'day_of_week', 'abnormally', 'access', 'accessed', 'ace',
       'action', 'ad', 'add',
       ...
       'used', 'user', 'validating', 'value', 'vendor', 'verify', 'version',
       'whether', 'window', 'write'],
      dtype='object', length=280)

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm thử
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=21)

In [ ]:
# Chuẩn hóa các cột là số
scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

In [ ]:
# Xem lại dữ liệu sau khi chuẩn hóa
print(X_train.head())

             id      hour  day_of_week  machine_name_DESKTOP-R5JMQHG  \
15982  0.025246 -0.047256    -1.155493                         False   
7260  -0.956148 -0.345256    -0.157865                          True   
12645 -0.508974  0.250743    -1.654307                         False   
82    -1.555926  1.144741     0.839763                         False   
18380  0.306439 -0.047256    -0.656679                         False   

       machine_name_DESKTOP-SEJ28PM  machine_name_DESKTOP-U66O8IT  \
15982                         False                         False   
7260                          False                         False   
12645                         False                         False   
82                            False                         False   
18380                         False                         False   

       machine_name_LAPTOP-1MKMTVPM  machine_name_TMP249-G3-M  \
15982                          True                     False   
7260                  

In [ ]:
# Khởi tạo mô hình SVM
clf = svm.SVC(kernel="rbf", gamma=0.05, C=0.01)
clf.fit(X_train, y_train)

SVC(C=0.01, gamma=0.05)

In [ ]:
y_pred = clf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.98
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      8013
           1       1.00      0.84      0.91       663
           2       1.00      0.86      0.92       827

    accuracy                           0.98      9503
   macro avg       0.99      0.90      0.94      9503
weighted avg       0.98      0.98      0.98      9503



In [ ]:
y_train_pred = clf.predict(X_train)
print(f'Accuracy: {accuracy_score(y_train, y_train_pred):.2f}')

Accuracy: 0.98
